
### Supervised Learning
### Activity: Building a Student Intervention System

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail or pass. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** 

This is a classification problem, Because the target variable "Passed" is catogorical variable. it has two possible outputs, yes or no.

### Question-2
load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [7]:
# Read student data
data = pd.read_csv("G:\study\Data Science - ICT\Week11\student-data.csv")

In [8]:
data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,no,no,5,5,4,4,5,4,11,no
391,MS,M,17,U,LE3,T,3,1,services,services,...,yes,no,2,4,5,3,4,2,3,yes
392,MS,M,21,R,GT3,T,1,1,other,other,...,no,no,5,5,3,3,3,3,3,no
393,MS,M,18,R,LE3,T,3,2,services,other,...,yes,no,4,4,1,3,4,5,0,yes


### Question-3
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [13]:
# Calculate number of students
n_students = data.shape[0]
n_students
#395 students

395

In [17]:
# Calculate number of features
n_features = data.shape[1]
n_features
#31 features

31

In [19]:
# Calculate passing students
n_passed = data['passed'].value_counts()[0]
n_passed
#265 students passed

265

In [20]:
# Calculate failing students
n_failed = data['passed'].value_counts()[1]
n_failed
#130 students failed

130

In [21]:
# Calculate graduation rate
grad_rate = (n_passed/n_students) * 100
grad_rate

67.08860759493672

In [38]:
# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students passed: {}".format(n_passed))
print ("Number of students failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 31
Number of students passed: 265
Number of students failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
you will prepare the data for modeling, training and testing.

### Question-4 Identify feature and target columns


separate the student data into feature and target columns to see if any features are non-numeric.

In [ ]:
# Extract feature columns

In [28]:
feature_cols = list(data.columns[:-1])
feature_cols

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences']

In [ ]:
# Extract target column 'passed'

In [36]:
target_col = data.columns[-1]
target_col

'passed'

In [ ]:
# Separate the data into feature data and target data (X and y, respectively)

In [37]:
X = data[feature_cols]
y = data[target_col]

### Question-5 Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [52]:
def preprocess_features(X):
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X = preprocess_features(X)

print('Total number of processed feature columns are : {}'.format(len(X.columns)))

Total number of processed feature columns are : 48


In [51]:
print ("Processed feature columns are :\n{}".format(list(X.columns)))

Processed feature columns are :
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Question - 6 Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. you will need to implement the following:
- Randomly shuffle and split the data (`X`, `y`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [45]:
# splitting the data into train and test
from sklearn.model_selection import train_test_split

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20)

In [50]:
# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print( "Testing set has {} samples.".format(X_test.shape[0]))

Training set has 316 samples.
Testing set has 79 samples.


### Question - 7  Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data and measure the accuracy score.

I will choose Random Forest, Adaboost and Logistic Regression.

###  Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

#explanation


1. Random Forest :

a. This classifier is used in fileds like credit scoring, crime risk, medical diagnosis, failure prediction etc.
b. Not prone to overfit and not computationally expensive which is what we want, to take less time.
c. They’re not easily interpretable.
d. Random forest are not prone to overfit as we have lots features containing integers, categorical and binary. 
e. This eliminates the risk of overfitting and not computationally expensive as advised by supervisors.


2. AdaBoost

This classifier is used in fileds biology, speech processing, computer vision etc.
Less tuning of parameters required and computational friendly.
Sensitive to noisy data and outliers.


3. Logistic Regression

Used in Financial forecasting, Software cost prediction, software effort prediction and software quality assurance, etc
It doesn't assume a linear relationship between the features and target variable. (Good for us as we might have with lots of features)
It requires large sample size in order to perform well. (Sad we don't have it but let's try.

In [72]:
# Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score


In [64]:
# fit model-1  on traning data 
#Random Forest
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

In [67]:
# predict on the test data 
y_pred=rf.predict(X_test)

In [66]:
# calculate the accuracy score
print("Accuracy is: ", accuracy_score(y_test,y_pred))

Accuracy is:  0.6835443037974683


In [ ]:
# fit the model-2 on traning data and predict on the test data and measure the accuracy

In [69]:
#AdaBoost
ab = AdaBoostClassifier()
ab.fit(X_train, y_train)

y_pred = ab.predict(X_test)
print("Accuracy is: ", accuracy_score(y_test,y_pred))

Accuracy is:  0.6582278481012658


In [ ]:
# fit the model-3 on traning data and predict on the test data and measure the accuracy

In [71]:
#Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print("Accuracy is: ", accuracy_score(y_test,y_pred))

Accuracy is:  0.7468354430379747


C:\Users\munee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Inference:

## The Accuracy is highest for 'Logistic Regression' (0.74). Hence it is the best model for Predictions in the given dataset. 